## OCR Notebook
Optical Character Recognition (OCR), is one of the most crucial tasks in document processing. The primary goal is to extract text and layout information in the form of bounding boxes. There are several popular service providers for OCR, including open source libraries, for more custom/private solutions. In this notebook, we will cover:
- OCR with AWS Textract
- OCR with GCP Document AI
- OCR with Tesseract (Open-Source)

In addition to demonstrating how to execute OCR processing jobs with DocPrompt, this notebook will also explore the powerful `DocumentProvenanceLocator`, a Rust implemented search tool which is powered by the OCR results.

### Environment Setup

To start, let's make sure the environment is setup correctly. Depending on what service provider you are using, there are a few environment variables you will need to set or you may choose to pass the credentials as kwargs at run-time instead.

**For GCP Document AI**:
- `GCP_SERVICE_ACCOUNT_FILE`: The path to the GCP Service Account JSON file
- `GCP_PROJECT_ID`: The GCP project you have provisioned for OCR tasks
- `GCP_PROCESSOR_ID`: The GCP provdier you have provisioned for OCR tasks
_For more info on setting up Document AI with GCP, please review documentation [here](https://cloud.google.com/document-ai/docs/setup)._

**For AWS Textract**:
- `AWS_ACCESS_KEY_ID`: The access key for your AWS IAM role with Textract Permissions
- `AWS_SECRET_ACCESS_KEY`: The secret access key for your AWS IAM role with Textract Permissions
- `AWS_DEFUALT_REGION`: The region that you wish to utilize
- `AWS_SESSION_TOKEN`: Can be used in lieu of the above credentials, to authenticate an AWS session.

**For Tesseract**:

You simply need to ensure that you have the `tesseract` library installed. For more information or to install tesseract on your local machine, review their documentation [here](https://tesseract-ocr.github.io/tessdoc/Installation.html)

In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")

# Checking GCP credentials 
GCP_SERVICE_ACCOUNT_FILE = os.environ.get("GCP_SERVICE_ACCOUNT_FILE", None)
GCP_PROJECT_ID = os.environ.get("GCP_PROJECT_ID", None)
GCP_PROCESSOR_ID = os.environ.get("GCP_PROCESSOR_ID", None)

assert GCP_SERVICE_ACCOUNT_FILE is not None
assert GCP_PROJECT_ID is not None
assert GCP_PROCESSOR_ID is not None


# Checking AWS credentials
AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID", None)
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY", None)
AWS_DEFAULT_REGION = os.environ.get("AWS_DEFAULT_REGION", None)

assert AWS_ACCESS_KEY_ID is not None
assert AWS_SECRET_ACCESS_KEY is not None
assert AWS_DEFAULT_REGION is not None

### Load Document Resources

With our environment properly configured, we can now begin loading our PDF documents into the environment. There are a few unique ways to store and retrieve documents, but we will opt for the simplest, out of the box method, which is the `load_document_node` utility. 

In [2]:
from docprompt import load_document_node

node = load_document_node("../data/example-1.pdf")
node

DocumentNode(document=PdfDocument(name='example-1.pdf', file_path='../data/example-1.pdf', password=None, page_count=42, document_hash='191e8a7d232bfdc773858c39a8ff6ac7'), metadata={})

We now have our PDF document loaded into memory as a document node. A `DocumentNode` is the container uitlized by DocPrompt for attaching metdata and processing task results directly to a PDF file. With our node setup, let's move onto running some OCR jobs.

### Amazon Textract OCR

As a first task, let's utilize Amazon Textract to run the OCR job on our document node. Running a task in docprompt takes three actions, which can be executed in three sequential lines of code:

1. Create your factory
2. Get your task provider
3. Execute your task

This simple call pattern is reflected across all tasks and all providers in the DocPrompt ecosystem. Let's see how simple this is with AWS textract:

In [11]:
from docprompt.tasks.factory import AmazonTaskProviderFactory

# Create your factory -- this will use the credentials we have established
# as .env variables above. However, you may also pass `kwargs` to the 
# factory at runtime, to override and `.env` variables you may have set.
aws_factory = AmazonTaskProviderFactory()

# Get your OCR provider -- for some factories, this method may require some
# positional arguments, in order to configure the task provider. We will see 
# this later on.
aws_ocr_provider = aws_factory.get_page_ocr_provider()

# Execute the OCR task on our document node -- this final step will execute the 
# OCR task on the document node. THe results will automatically be persisted 
# to the document nodes internal state.
aws_results = await aws_ocr_provider.aprocess_document_node(node)

Processing document: 100%|██████████| 42/42 [00:10<00:00,  4.09it/s]


Our results are indexed by page number. Let's take a peek at the results from a couple random pages:

In [12]:
# Results from page 1
aws_results[1]

OcrPageResult(provider_name='aws_textract', when=datetime.datetime(2024, 7, 2, 12, 20, 10, 458672), page_text='IN THE SUPERIOR COURT OF FULTON COUNTY STATE OF GEORGIA , PLAINTIFF, VS. CIVIL ACTION FILE NO: PATIENCE AJUZIE, DEFENDANT. The following deposition of ROBERT D. COSTON, Ph. D. was taken pursuant to stipulations contained herein, the reading and signing of the deposition waived; before Tanga Donnelly, Certified Court Reporter in the State of Georgia, on Friday, January 31, 2014 at 2719 Buford Highway, NE, Atlanta, Georgia; commencing at 1:30 p.m. M MAB R METRO ATLANTA REPORTERS INC. PO BOX 1442 Snellville, Georgia 30078 770-985-2344 WWW.METROATLANTAREPORTERS.COM 1', extra={})

In [13]:
# Results from page 16
aws_results[16]

OcrPageResult(provider_name='aws_textract', when=datetime.datetime(2024, 7, 2, 12, 20, 10, 533654), page_text="1 BY MR. HIESTAND: (Resuming) 2 Q. Of these two parts, would D have been the 3 one that was more relevant to you in developing your 4 opinions or conclusions, or maybe both of them? 5 A. Well, they both were and I read both of 6 them, but I am asked to determine the discounted 7 value of the life care plan itself and the second 8 part is the life care plan. 9 Q. So in other words, when you were utilizing 10 figures provided by Ms. Willard to develop your 11 opinions and conclusions, it would have come from 12 Exhibit D? 13 A. Yes. 14 Q. I'm not expecting the answer to this to be 15 yes, but did you ever review any other life care 16 plans from any other persons other than Ms. Willard 17 in developing your opinions and conclusions? 18 A. In this case? 19 O. Yes, sir. . 20 A. No. This is the only life care plan I 21 have. 22 Q. Roughly, this will be a very rough figure, 23 how m

As we can see, the text of this legal deposition has been extracted by AWS textract and is ready for downstream processing. Now that we have seen some example results, let's run a second OCR task with GCP Document AI.

### GCP Document AI

The general pattern for setting up an OCR task with GCP is largely identical to AWS textract, with a few small caveats. Since GCP Document AI requires users to provision their own endpoint before they can access the service, we must provide additional parameters to the `get_page_ocr_provider` method of the GCP Proivder Factory, in order to allow DocPrompt to configure a proper connection with GCP's Document AI service.

Additionally, in this example, we will also modify some of the `kwargs` of the `aprocess_document_node` task, to see some of the customizable funcitonality that docprompt provides.

In [14]:
from docprompt.tasks.factory import GCPTaskProviderFactory

# Create your factory -- this will use the credentials we have established
# as .env variables above. However, you may also pass `kwargs` to the 
# factory at runtime, to override and `.env` variables you may have set.
gcp_factory = GCPTaskProviderFactory()

# Get your OCR provider -- here, we need to specifcy the GCP project and processor 
# ID's so that DocPrompt can correctly intialize a connection to the Document AI service.
gcp_ocr_provider = gcp_factory.get_page_ocr_provider(GCP_PROJECT_ID, GCP_PROCESSOR_ID)

# Run OCR task -- here, since we have already populated our OCR results cache with our 
# AWS Textract job over the entire document, we do not need to save these additional
# OCR results to the results cache on the document as well.
gcp_results = gcp_ocr_provider.process_document_node(
    node,
    contribute_to_document=False # Don't save the results to the node, just return them
)

100%|██████████| 3/3 [00:02<00:00,  1.31it/s]


_NOTE: You may notice the discrepency in the progress bars. This is due to the way that GCP process PDFs vs. AWS. The Document AI endpoints takes the raw file bytes of the PDF file and receieves them in chunks of N pages, while AWS requires a single rasterized image of a PDF for each OCR request._

We should be able to see that looking at the results from GCP, we have fairly similar sets of extracted text.

In [15]:
# Results from page 1
gcp_results[1]

OcrPageResult(provider_name='gcp_documentai', when=datetime.datetime(2024, 7, 2, 12, 20, 22, 429505), page_text='VS.\nIN THE SUPERIOR COURT OF FULTON COUNTY\nSTATE OF GEORGIA\nPLAINTIFF,\nPATIENCE AJUZIE,\nDEFENDANT.\n*\nCIVIL ACTION FILE NO:\nThe following deposition of ROBERT D.\nCOSTON, Ph.D. was taken pursuant to\nstipulations contained herein, the reading\nand signing of the deposition waived;\nbefore Tanga Donnelly, Certified Court\nReporter in the State of Georgia, on\nFriday, January 31, 2014 at 2719 Buford\nHighway, NE, Atlanta,\nat 1:30 p.m.\nGeorgia; commencing\nM\nR\nA\nMETRO ATLANTA REPORTERS INC.\nPO BOX 1442\nSnellville, Georgia 30078\n770-985-2344\nWWW.METROATLANTAREPORTERS.COM\n1\n', extra={'quality_score': 0.0, 'defect_scores': {}})

In [16]:
# Results from page 16
gcp_results[16]

OcrPageResult(provider_name='gcp_documentai', when=datetime.datetime(2024, 7, 2, 12, 20, 23, 285075), page_text="1\n2\n3\n4\n5\nBY MR. HIESTAND: (Resuming)\nQ. Of these two parts, would D have been the\none that was more relevant to you in developing your\nopinions or conclusions,\nor maybe both of them?\nA. Well, they both were and I read both of\n6 them, but I am asked to determine the discounted\nvalue of the life care plan itself and the second\nထ\n9\n10\npart is the life care plan.\nQ. So in other words, when you were utilizing\nfigures provided by Ms. Willard to develop your\n11 opinions and conclusions, it would have come from\n12\nExhibit D?\n13\nA.\nYes.\n14\n15\n16\n17\nQ. I'm not expecting the answer to this to be\nyes, but did you ever review any other life care\nplans from any other persons other than Ms. Willard\nin developing your opinions and conclusions?\n18\nA.\nIn this case?\n19\nQ.\nYes, sir.\n20\nA.\nNo. This is the only life care plan I\n21\nhave.\n22\nQ.\n23\n24\

As we can see, we have created the same set of OcrPageResults, just with a different provider to produce the results.

### Tesseract Implementation

When data privacy or customization is a top priority, being able ot utilize a custom open source model such as Tesseract for various document processing tasks is essential. The final guide in this notebook shows how to utilize Tessearact as your open source provider.

_Note: You must be sure to install tesseract before this provider will be functional on your machine._

In [20]:
from docprompt.tasks.ocr.tesseract import TesseractOcrProvider

ocr_provider = TesseractOcrProvider()
tesseract_results = ocr_provider.process_document_node(node)

In [21]:
# Page 1 Results
tesseract_results[1]

OcrPageResult(provider_name='tesseract', when=datetime.datetime(2024, 7, 2, 13, 50, 43, 985935), page_text='IN THE SUPERIOR COURT OF FULTON COUNTY STATE OF GEORGIA PLAINTIFF, vs. CIVIL ACTION FILE NO: PATIENCE AJUZIE, DEFENDANT. The following deposition of ROBERT D. COSTON, Ph.D. was taken pursuant to stipulations contained herein, the reading and signing of the deposition waived; before Tanga Donnelly, Certified Court Reporter in the State of Georgia, on Friday, January 31, 2014 at 2719 Buford Highway, NE, Atlanta, Georgia; commencing at 1:30 p.m. MHR MerRo ATLANTA REPORTERS INC. PO BOX 1442 Snellville, Georgia 30078 770-985-2344 WWW.METROATLANTAREPORTERS.COM', extra={})

In [22]:
# Page 16 Results
tesseract_results[16]

OcrPageResult(provider_name='tesseract', when=datetime.datetime(2024, 7, 2, 13, 50, 45, 271876), page_text="10 ql 12 13 14 15 16 17 18 19 20 al 22 23 24 25 BY MR. HIESTAND: (Resuming) Q. Of these two parts, would D have been the one that was more relevant to you in developing your opinions or conclusions, or maybe both of them? Well, they both were and I read both of them, but I am asked to determine the discounted value of the life care plan itself and the second part is the life care plan. \n Q. So in other words, when you were utilizing figures provided by Ms. Willard to develop your opinions and conclusions, it would have come from Exhibit D? \n A. Yes. \n Q. I'm not expecting the answer to this to be yes, but did you ever review any other life care plans from any other persons other than Ms. Willard in developing your opinions and conclusions? \n A, In this case? \n Q. Yes, sir. A. No. This is the only life care plan I Roughly, this will be a very rough figure, how many of these t

All three providers exposed incredibly similar results. The `page_text` field that is displayed is a convience method for easily dumping all of the extracted text from an image into a formatted string. However, for more detailed parsing, the Bounding Boxes of the results should be used.